In [2]:
import sys
import os
import re
import json

import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
import pprint
# import psycopg2

import numpy as np

import plotly.graph_objects as go
import plotly.io as pio
# import colorlover as cl
from IPython.display import HTML
import time
import requests
import pytz
from datetime import timedelta 
from itertools import product

import xml.etree.ElementTree as parser

In [3]:
csv_path = '../data/board_info_csv/unprocessed/930-0011_2.F.2_RUG2_mainboard_Pick Place_Standard_Assembly files.out.csv'
# If problem with encoding, try:
# Open csv in visual studio code (or other editor) and save as utf-8
df_main_board = pd.read_csv(csv_path,sep='|')

In [4]:
df_main_board.head()

,Part Number,Comment,Description,Designator,Footprint,Layer,Ref-X(mm),Ref-Y(mm),Rotation
0,750-0001,0R,Resistor 0R 0402 1%,R124_1,R0402-IPC-MD,BottomLayer,126.0904,165.4081,90
1,750-0001,0R,Resistor 0R 0402 1%,R123_1,R0402-IPC-MD,BottomLayer,127.0053,165.1637,90
2,730-0015,10uF,MLCC 10uF 0805 10V 10% X7R,C985AL_1,C0805-IPC-MD,BottomLayer,64.4934,50.1591,180
3,730-0015,10uF,MLCC 10uF 0805 10V 10% X7R,C985AH_1,C0805-IPC-MD,BottomLayer,47.2264,49.9767,180
4,730-0016,10nF,MLCC 10nF 0402 50V 10% X7R,C984AL_1,C0402-IPC-MD,BottomLayer,61.3575,50.2107,90


In [5]:
df_main_board[df_main_board['Designator']=='U911U5_1']

,Part Number,Comment,Description,Designator,Footprint,Layer,Ref-X(mm),Ref-Y(mm),Rotation
771,770-0113,ADUM1250ARZ-RL7,"Hot Swappable, Dual I2C Isolators 2500Vrms",U911U5_1,SOIC127P600X175-8N,TopLayer,69.9589,47.5992,270


In [6]:
df_top=df_main_board[df_main_board['Layer']=='TopLayer'].copy()
df_top

,Part Number,Comment,Description,Designator,Footprint,Layer,Ref-X(mm),Ref-Y(mm),Rotation
537,750-0012,100k,Resistor 100 k 0402 1% 1/16W,R926U5_1,R0402-IPC-MD,TopLayer,68.788500,34.499200,0
538,750-0012,100k,Resistor 100 k 0402 1% 1/16W,R926U4_1,R0402-IPC-MD,TopLayer,88.205600,34.328700,0
539,750-0012,100k,Resistor 100 k 0402 1% 1/16W,R926U3_1,R0402-IPC-MD,TopLayer,106.079500,34.348000,0
540,750-0012,100k,Resistor 100 k 0402 1% 1/16W,R926U2_1,R0402-IPC-MD,TopLayer,124.014600,34.674900,0
541,750-0012,100k,Resistor 100 k 0402 1% 1/16W,R926U1_1,R0402-IPC-MD,TopLayer,142.300000,34.900000,0
...,...,...,...,...,...,...,...,...,...
1132,700-0013,pin_paste,pin_paste,pin_paste_P5_1_20,0805,TopLayer,104.890099,24.927933,360
1133,700-0013,pin_paste,pin_paste,pin_paste_P5_1_21,0805,TopLayer,107.430104,19.287927,360
1134,700-0013,pin_paste,pin_paste,pin_paste_P5_1_22,0805,TopLayer,107.430104,24.927933,360
1135,700-0013,pin_paste,pin_paste,pin_paste_P5_1_23,0805,TopLayer,109.970109,19.287927,360


In [10]:
draw_comps = go.Scattergl(x=df_top['Ref-X(mm)'], 
                        y=df_top['Ref-Y(mm)'],
                        mode='markers',
                        text=df_top.Designator
                        )
draw_comps_2 = go.Scattergl(x=df_top.iloc[1059:1060]['Ref-X(mm)'], 
                         y=df_top.iloc[1059:1060]['Ref-Y(mm)'],
                        mode='markers'
                        )
data = [draw_comps, draw_comps_2]

# Useful layout
layout = go.Layout(
    ## title='example of 7778, rug 502',
    legend=dict(orientation="v", x=0.2, y=1),
    yaxis=dict(
        title='rx_vs_time'
    ),
    xaxis=dict(
       # title='km SNCF',
       # dtick=1
       #  range=[90, 100.05]

       tickangle = 45,
    ),
    yaxis2=dict(
                title='railster state',
                titlefont=dict(
                    color='rgb(148, 103, 255)'
                ),
                tickfont=dict(
                    color='rgb(148, 103, 255)'
                ),
                overlaying='y',
                side='right'
    ),
    barmode='group',
   #yaxis_type="log"
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height=700, width=700)
fig.show()

In [30]:
all_the_foot_prints = list(df_main_board.Footprint.unique())
dict_body_X = {}
dict_body_Y = {}
dict_glue_X = {}
dict_glue_Y = {}
counter = 0

# Checking package.xml files 
for a_foot_print in all_the_foot_prints:
    try:
        tree = parser.parse('../data/Package/{}.package.xml'.format(a_foot_print))
        root = tree.getroot()
        for child in root:
            #if not child.tag=='ChipPackage':
            #    continue
            for a_sub_child in child:
                if  a_sub_child.tag=='GeometryDescription':
                    for a_sub_sub_chil in a_sub_child:
                        if  a_sub_sub_chil.tag =='BodyX':
                            dict_body_X[a_foot_print]=a_sub_sub_chil.attrib['NomValue']
                        if a_sub_sub_chil.tag =='BodyY':
                            dict_body_Y[a_foot_print]=a_sub_sub_chil.attrib['NomValue']
                elif child.tag == 'DispPattern':
                    for a_sub_child in child:
                        if not a_sub_child.tag == 'DispObjectArray':
                            continue
                        the_x=[]
                        the_y=[]
                        for a_sub_sub in a_sub_child:
                            if not a_sub_sub.tag == 'DispDot':
                                continue
                            for a_sub_sub_sub in a_sub_sub:
                                if not a_sub_sub_sub.tag=='Position':
                                    continue
                                the_x.append(float(a_sub_sub_sub.attrib['X_value']))
                                the_y.append(float(a_sub_sub_sub.attrib['Y_value']))
                        dict_glue_X[a_foot_print]=max(the_x) - min(the_x)
                        dict_glue_Y[a_foot_print]=max(the_y) - min(the_y)
    except KeyboardInterrupt:
        break
    except:
        counter +=1
        print('pb with', a_foot_print)
print(counter)

pb with QFN50P500X700X80_HS-39
pb with FOX_measure_point_3mm
pb with Fiducial_1mm
pb with F-TE-R18-H
pb with V-solder_pad_1x1
pb with WE-LF_SMD_Type_SH
pb with CCHIM-D10XL30XE5mm-HI
pb with MGDDI-20_single
pb with CON-REC-RIGHT-F2X4-P0.1IN-IPL1
pb with 0805
10


In [31]:
# Adding the columns
df_main_board['dimX'] = df_main_board['Footprint'].map(dict_body_X)
df_main_board['dimY'] = df_main_board['Footprint'].map(dict_body_Y)
df_main_board['dimglueX'] = df_main_board['Footprint'].map(dict_glue_X)
df_main_board['dimglueY'] = df_main_board['Footprint'].map(dict_glue_Y)

In [32]:
# Adding new columns, max of the two
df_main_board['max_X'] = df_main_board.apply(lambda x: max([float(x['dimX']), x['dimglueX']]), axis=1)
df_main_board['max_Y'] = df_main_board.apply(lambda x: max([float(x['dimY']), x['dimglueY']]), axis=1)

In [34]:
# Saving the csv
df_main_board.to_csv('../data/board_info_csv/processed/board_data_2F2.csv', index=False)